<a href="https://colab.research.google.com/github/intelligenxe/FinRL-type/blob/main/FinRL_Data_Wrangle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These codes are based on the codes of the following videos series by linkedin.com/in/everett-minshall/ : <br>
https://www.youtube.com/watch?v=OrqffjqkklM  <br>
https://www.youtube.com/watch?v=uYC3sc5gers <br>
https://www.youtube.com/watch?v=335QxO36pzo&t=15s <br>
The codes in the above videos are in turn based on https://github.com/AI4Finance-Foundation/FinRL?tab=readme-ov-file#tutorials (FinRL Tutorials).

# Install and Import Packages

In [ ]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.23
    Uninstalling SQLAlchemy-2.0.23:
      Successfully uninstalled SQLAlchemy-2.0.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.51 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.6 MB/s eta 0:00:00
  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-s4dnsty4
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-s4dnsty4
  Resolved http

In [ ]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

In [ ]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2023-05-01'

In [ ]:
symbols = [
    'aapl',
    'msft',
    'meta',
    'ibm',
    'hd',
    'cat',
    'amzn',
    'intc',
    't',
    'v'
    'gs'
]

In [ ]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TRADE_END_DATE,
                                ticker_list = symbols).fetch_data()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
Shape of DataFrame:  (32958, 8)


In [ ]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.747390,746015200,aapl,4
1,2009-01-02,2.567500,2.726500,2.553500,2.718000,145928000,amzn,4
2,2009-01-02,44.910000,46.980000,44.709999,31.093393,7117200,cat,4
3,2009-01-02,23.070000,24.190001,22.959999,16.649143,14902500,hd,4
4,2009-01-02,80.200768,83.738052,80.200768,50.100777,7905877,ibm,4


# Preprocess Data

In [ ]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

Successfully added technical indicators
[*********************100%%**********************]  1 of 1 completed
Shape of DataFrame:  (3604, 8)
Successfully added vix
Successfully added turbulence index


In [ ]:
processed

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,3.067143,3.251429,3.041429,2.747390,746015200,aapl,4,0.000000,2.969346,2.641385,100.000000,66.666667,100.000000,2.747390,2.747390,39.189999,0.000000
1,2009-01-02,2.567500,2.726500,2.553500,2.718000,145928000,amzn,4,0.000000,2.969346,2.641385,100.000000,66.666667,100.000000,2.718000,2.718000,39.189999,0.000000
2,2009-01-02,44.910000,46.980000,44.709999,31.093393,7117200,cat,4,0.000000,2.969346,2.641385,100.000000,66.666667,100.000000,31.093393,31.093393,39.189999,0.000000
3,2009-01-02,23.070000,24.190001,22.959999,16.649143,14902500,hd,4,0.000000,2.969346,2.641385,100.000000,66.666667,100.000000,16.649143,16.649143,39.189999,0.000000
4,2009-01-02,80.200768,83.738052,80.200768,50.100777,7905877,ibm,4,0.000000,2.969346,2.641385,100.000000,66.666667,100.000000,50.100777,50.100777,39.189999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28827,2023-04-27,288.000000,294.709991,287.799988,288.043152,3133600,hd,3,0.867125,296.724969,279.499536,48.591920,15.429901,7.315064,285.132974,291.722891,17.030001,5.475575
28828,2023-04-27,126.370003,127.019997,125.459999,122.448189,3204900,ibm,3,-0.792540,128.409098,119.511597,44.911359,-38.402377,1.220107,123.207804,124.897304,17.030001,5.475575
28829,2023-04-27,28.750000,29.950001,28.510000,29.534502,60186200,intc,3,-0.018806,33.840405,28.574721,50.727392,-66.345036,13.482737,30.502171,28.709950,17.030001,5.475575
28830,2023-04-27,295.970001,305.200012,295.250000,302.907654,46462600,msft,3,5.108002,297.163501,274.137839,63.072947,228.799245,42.136850,281.995372,268.954163,17.030001,5.475575


In [ ]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [ ]:
processed_full

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,aapl,3.067143,3.251429,3.041429,2.747390,746015200.0,4.0,0.000000,2.969346,2.641385,100.000000,66.666667,100.000000,2.747390,2.747390,39.189999,0.000000
1,2009-01-02,amzn,2.567500,2.726500,2.553500,2.718000,145928000.0,4.0,0.000000,2.969346,2.641385,100.000000,66.666667,100.000000,2.718000,2.718000,39.189999,0.000000
2,2009-01-02,cat,44.910000,46.980000,44.709999,31.093393,7117200.0,4.0,0.000000,2.969346,2.641385,100.000000,66.666667,100.000000,31.093393,31.093393,39.189999,0.000000
3,2009-01-02,hd,23.070000,24.190001,22.959999,16.649143,14902500.0,4.0,0.000000,2.969346,2.641385,100.000000,66.666667,100.000000,16.649143,16.649143,39.189999,0.000000
4,2009-01-02,ibm,80.200768,83.738052,80.200768,50.100777,7905877.0,4.0,0.000000,2.969346,2.641385,100.000000,66.666667,100.000000,50.100777,50.100777,39.189999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41827,2023-04-27,hd,288.000000,294.709991,287.799988,288.043152,3133600.0,3.0,0.867125,296.724969,279.499536,48.591920,15.429901,7.315064,285.132974,291.722891,17.030001,5.475575
41828,2023-04-27,ibm,126.370003,127.019997,125.459999,122.448189,3204900.0,3.0,-0.792540,128.409098,119.511597,44.911359,-38.402377,1.220107,123.207804,124.897304,17.030001,5.475575
41829,2023-04-27,intc,28.750000,29.950001,28.510000,29.534502,60186200.0,3.0,-0.018806,33.840405,28.574721,50.727392,-66.345036,13.482737,30.502171,28.709950,17.030001,5.475575
41830,2023-04-27,msft,295.970001,305.200012,295.250000,302.907654,46462600.0,3.0,5.108002,297.163501,274.137839,63.072947,228.799245,42.136850,281.995372,268.954163,17.030001,5.475575


# Save the Data

In [ ]:
# Split the data
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

23144
5688


In [ ]:
from google.colab import drive

drive.mount('/content/drive')
# the author of the video had this path = '/content/drive/My Drive/Colab Notebooks/QuantFinance/FinRL/Data/train_data.csv'
# and this                         path = '/content/drive/My Drive/Colab Notebooks/QuantFinance/FinRL/Data/trade_data.csv'
train_path = '/content/drive/My Drive/Colab Notebooks/FinRL_type/data/train_data.csv'
trade_path = '/content/drive/My Drive/Colab Notebooks/FinRL_type/data/trade_data.csv'

with open(train_path, 'w', encoding = 'utf-8-sig') as f:
  train.to_csv(f)

with open(trade_path, 'w', encoding = 'utf-8-sig') as f:
  trade.to_csv(f)

Mounted at /content/drive


# Consider Joining the Newsleter

https://witty-motivator-1414.ck.page/acb393f729